# Cooling Effect Module Draft 

In [1]:
# Install the GUS framework.

import sys
!{sys.executable} -m pip install --index-url https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple gus==0.1.7

Looking in indexes: https://test.pypi.org/simple/, https://pypi.org/simple


In [2]:
import numpy as np
import pandas as pd

In [3]:
DBH=15
TH=5
TCD=4
TCDth=3

In [4]:
def compute_leaf_area_index(
    dbh,
    tree_height,
    crown_depth,
    crown_diameter,
    crown_missing=0,
    shade_factor=0.85,
):
    """The function given allometrics of a tree computes its leaf, bark and plant area indices.

    Args:
        dbh: (:obj:`float`): the diameter in cm of the trunk usually measured at 1.3m from the ground.
        tree_height: (:obj:`float`): The tree height in meters.
        crown_debth: (:obj:`float`): The vertical length of tree crown in meters.
        crown_diameter: (:obj:`float`): The horizontal length (diameter) of tree crown in meters.
        crown_missing: (:obj:`float`): The percentage loss of the crown.
        shade_factor: (:obj:`float`): The percentage of sky covered by foliage and branches.

        Shade factor, is the percentage of sky covered by foliage and branches within the perimeter of individual tree crowns,
        can vary by species from about 60% to 95% when trees are in-leaf (McPherson, 1984). The value below is set according to 
        Glasgow mean and needs to be updated per city or per tree.

    Returns:
        (:obj:`float`): It eturns the LAI.
    Note:
        The beta multipliers and the main equation is based on Nowak (1996).

    TODO:
        Parametrize beta multipliers.
    """
    loss = crown_missing
    th = tree_height
    cw = crown_diameter
    cd = crown_depth
    sf = shade_factor
    beta_0 = -4.3309
    beta_1 = 0.2942
    beta_2 = 0.7312
    beta_3 = 5.7217
    beta_4 = 0.0148

    def compute_under_canopy_area(crown_diameter):
        return pow((crown_diameter / 2), 2) * np.pi

    # Outer surface area estimate below is based on Gacka-Grzesikiewicz (1980).
    under_canopy = compute_under_canopy_area(cw)
    crown_surface = np.pi * cw * (cd + cw) / 2
    leaf_area = (1 - loss) * np.exp(
        beta_0 + beta_1 * th + beta_2 * cw + beta_3 * sf - beta_4 * crown_surface
    )
    leaf_area_index = leaf_area / under_canopy
    return leaf_area_index

In [5]:
compute_leaf_area_index(dbh=DBH, tree_height=TH, crown_diameter=TCD, crown_depth=TCDth)

5.733824472020663

In [6]:
def compute_gross_canopy_index(
    tree_height,
    crown_depth,
    crown_diameter
):
    """The function given allometrics of a tree computes an index to account shading effect immediately underneath its canopy.

    Args:
        tree_height: (:obj:`float`): The tree height in meters.
        crown_height: (:obj:`float`): The vertical length of tree crown in meters.
        crown_width: (:obj:`float`): The horizontal length (diameter) of tree crown in meters.

    Returns:
        (:obj:`float`): returns the GCI index
    Note:
        The index is based on (Zhang et.al., 2020). It accounts for the overall immediate shading effect of a tree.
        It accounts both crown diameter (tw) and the crown base height (th - ch), the distance from the ground to the start of
        the canopy.
    """
    th = tree_height
    cw = crown_diameter
    cd = crown_depth
    crown_base_height = th  - cd
    if crown_base_height < 0.1:
        crown_base_height = 0.1 
    GCI = cw / crown_base_height
    return GCI

In [7]:
compute_gross_canopy_index(tree_height = TH, crown_depth = TCDth, crown_diameter = TCD)

2.0

In [8]:
def compute_crown_volume_index(
    leaf_area_index,
    crown_canopy_index
):
    """The function given computes a composite index taking into account both leaf density and canopy shape.

    Args:
        leaf_area_indext: (:obj:`float`): The index on the leaf density per the area under the canopy.
        crown_canopy_index: (:obj:`float`): The index that accounts for the geomtric shape of the canopy relevant to shading.
        
    Returns:
        (:obj:`float`): returns the CVI index
    Note:
        The index is adopted from (Zhang et.al., 2020).
    """
    return leaf_area_index * crown_canopy_index

In [9]:
LAI = compute_leaf_area_index(DBH,TH,TCD,TCDth)
GCI = compute_gross_canopy_index(TH,TCDth,TCD)
compute_crown_volume_index(LAI,GCI)

11.547033684456904

In [10]:
df = pd.read_csv("trees_yearly.csv")
df.set_index("AgentID", inplace=True)
df.rename(columns={"Step": "year"}, inplace=True)
df.tail(20)

,year,species,dbh,height,crownH,crownW,canopy_overlap,cle,condition,dieback,biomass,seq,carbon,deroot,detrunk,mulched,burnt,coordinates
AgentID,,,,,,,,,,,,,,,,,,
153,50,decidu,30.439456,15.931190,9.125725,8.037718,0.139470,0.939746,excellent,0.000985,515.684843,20.717222,257.842422,0.0,0.0,0.012695,0.000000,"(6, 9)"
154,50,decidu,27.691566,15.133120,8.613116,7.557668,0.377894,0.844516,excellent,0.003000,416.796679,20.405725,208.398339,0.0,0.0,0.031260,0.000000,"(9, 8)"
155,50,decidu,24.806399,14.255167,8.037429,7.027223,0.509405,0.775526,excellent,0.002000,325.387001,15.288357,162.693500,0.0,0.0,0.000000,0.016269,"(9, 2)"
156,50,decidu,24.952244,14.300632,8.067451,7.054686,0.373618,0.831053,good,0.010953,329.707622,16.330511,164.853811,0.0,0.0,0.090280,0.000000,"(10, 4)"
157,50,decidu,24.461331,14.147108,7.966007,6.961970,0.483045,0.788265,excellent,0.001567,315.290348,15.263301,157.645174,0.0,0.0,0.012351,0.000000,"(4, 7)"
158,50,conifer,15.256274,9.171866,5.349862,3.472220,0.243380,0.873386,replaced,1.000000,98.620274,0.000000,49.310137,0.0,0.0,0.000000,0.000000,"(1, 5)"
159,50,decidu,19.784401,12.606888,6.943822,6.039380,0.696196,0.682728,excellent,0.004747,195.585539,10.087746,97.792769,0.0,0.0,0.023213,0.000000,"(3, 6)"
160,50,decidu,17.302682,11.721643,6.360461,5.520953,0.182742,0.917570,replaced,1.000000,144.661371,0.000000,72.330685,0.0,0.0,0.000000,0.000000,"(2, 10)"
161,50,conifer,15.965977,9.535299,5.520800,3.596589,0.114319,0.940309,excellent,0.007302,110.221784,5.618035,55.110892,0.0,0.0,0.020122,0.000000,"(13, 8)"


In [11]:
t_42 = df.xs(42)
t_31 = df.xs(31)
t_31.tail()

,year,species,dbh,height,crownH,crownW,canopy_overlap,cle,condition,dieback,biomass,seq,carbon,deroot,detrunk,mulched,burnt,coordinates
AgentID,,,,,,,,,,,,,,,,,,
31,46,decidu,49.904146,20.838819,11.759976,10.735481,0.346223,0.878366,excellent,0.003356,1568.577591,12.673916,784.288796,0.0,0.0,0.131604,0.000000,"(3, 9)"
31,47,decidu,50.255590,20.918409,11.791482,10.772873,0.348965,0.877084,excellent,0.004356,1593.544189,12.483299,796.772094,0.0,0.0,0.173537,0.000000,"(3, 9)"
31,48,decidu,50.608610,20.998100,11.822557,10.810027,0.349508,0.877037,excellent,0.005356,1618.843506,12.649659,809.421753,0.0,0.0,0.216764,0.000000,"(3, 9)"
31,49,decidu,50.947728,21.074413,11.851866,10.845337,0.350415,0.876797,excellent,0.004356,1643.355149,12.255822,821.677575,0.0,0.0,0.178962,0.000000,"(3, 9)"
31,50,decidu,51.242808,21.140629,11.876938,10.875758,0.352722,0.875722,excellent,0.005356,1664.850374,10.747613,832.425187,0.0,0.0,0.000000,0.222924,"(3, 9)"


In [12]:
t_42 = t_42[['year','species','dbh','height','crownH', 'crownW']]
t_31 = t_31[['year','species','dbh','height','crownH', 'crownW']]

In [13]:
t_31.head()

,year,species,dbh,height,crownH,crownW
AgentID,,,,,,
31,1,decidu,10.160808,8.778266,4.523209,3.917328
31,2,decidu,11.624329,9.443938,4.918861,4.259451
31,3,decidu,13.061136,10.061116,5.297682,4.588556
31,4,decidu,14.468776,10.636331,5.659583,4.904473
31,5,decidu,15.966213,11.220819,6.034541,5.233474


In [14]:
t_31['LAI'] = t_31.apply(lambda row: compute_leaf_area_index(row['dbh'], row['height'],row['crownH'], row['crownW']), axis=1)
t_31.head()

,year,species,dbh,height,crownH,crownW,LAI
AgentID,,,,,,,
31,1,decidu,10.160808,8.778266,4.523209,3.917328,15.203610
31,2,decidu,11.624329,9.443938,4.918861,4.259451,17.459500
31,3,decidu,13.061136,10.061116,5.297682,4.588556,19.836204
31,4,decidu,14.468776,10.636331,5.659583,4.904473,22.302067
31,5,decidu,15.966213,11.220819,6.034541,5.233474,25.050609


In [15]:
t_31['GPI'] = t_31.apply(lambda row: compute_gross_canopy_index(row['height'],row['crownH'], row['crownW']), axis=1)
t_31.head()

,year,species,dbh,height,crownH,crownW,LAI,GPI
AgentID,,,,,,,,
31,1,decidu,10.160808,8.778266,4.523209,3.917328,15.203610,0.920629
31,2,decidu,11.624329,9.443938,4.918861,4.259451,17.459500,0.941299
31,3,decidu,13.061136,10.061116,5.297682,4.588556,19.836204,0.963288
31,4,decidu,14.468776,10.636331,5.659583,4.904473,22.302067,0.985478
31,5,decidu,15.966213,11.220819,6.034541,5.233474,25.050609,1.009100


In [16]:
t_31['CVI'] = t_31.apply(lambda row: compute_crown_volume_index(row['LAI'],row['GPI']), axis=1)
t_31.head()

,year,species,dbh,height,crownH,crownW,LAI,GPI,CVI
AgentID,,,,,,,,,
31,1,decidu,10.160808,8.778266,4.523209,3.917328,15.203610,0.920629,13.996883
31,2,decidu,11.624329,9.443938,4.918861,4.259451,17.459500,0.941299,16.434609
31,3,decidu,13.061136,10.061116,5.297682,4.588556,19.836204,0.963288,19.107969
31,4,decidu,14.468776,10.636331,5.659583,4.904473,22.302067,0.985478,21.978188
31,5,decidu,15.966213,11.220819,6.034541,5.233474,25.050609,1.009100,25.278572


In [17]:

def compute_change(current_value, initial_value):
    """ The function to copmpute percentage change with respect to a given base value.
    """
    if initial_value == 0:
        return 0
    
    return ((current_value - initial_value) / initial_value) * 100

In [18]:
base_lai = list(t_31['LAI'])[0]
base_cvi = list(t_31['CVI'])[0]
base_gpi = list(t_31['GPI'])[0]
t_31['dLAI'] = t_31['LAI'].apply(lambda row: compute_change(row, base_lai))
t_31['dGPI'] = t_31['GPI'].apply(lambda row: compute_change(row, base_gpi))
t_31['dCVI'] = t_31['CVI'].apply(lambda row: compute_change(row, base_cvi))

In [19]:
t_31.tail()

,year,species,dbh,height,crownH,crownW,LAI,GPI,CVI,dLAI,dGPI,dCVI
AgentID,,,,,,,,,,,,
31,46,decidu,49.904146,20.838819,11.759976,10.735481,80.901747,1.182472,95.664082,432.121973,28.441797,583.467022
31,47,decidu,50.255590,20.918409,11.791482,10.772873,81.469421,1.180340,96.161584,435.855785,28.210130,587.021400
31,48,decidu,50.608610,20.998100,11.822557,10.810027,82.049072,1.178135,96.664875,439.668372,27.970652,590.617134
31,49,decidu,50.947728,21.074413,11.851866,10.845337,82.615258,1.175959,97.152155,443.392397,27.734302,594.098483
31,50,decidu,51.242808,21.140629,11.876938,10.875758,83.115740,1.174020,97.579541,446.684262,27.523686,597.151919


## Cooling Effect

### Calibration

In [20]:
dT_max = 0.67
m1 = 0.1
m2 = 0.75
m4 = 0.75
m5 = 0.2
T0 = 10
T1 = 18
T2 = 26
T3 = 35
T4 = 45
T5 = 50 
avg_conifer_cooling_factor = 0.66

def cooling_multiplier(T):
    if T < T0: return 0
    if T < T1: return m1 * dT_max
    if T < T2: return m2 * dT_max
    if T < T3: return dT_max
    if T < T4: return m4 * dT_max
    if T < T5: return m5 * dT_max
    return 0

def compute_cooling(canopy_change_cvi, T = 30, species = None):
        if species == 'conifer':
                species_factor = 0.6
        elif  species == 'decidu':
                species_factor = 0.9
        else:
                species_factor = 1
        return species_factor * np.sqrt(canopy_change_cvi/10) * cooling_multiplier(T)

In [21]:
cooling_multiplier(30)

0.67

In [22]:
grid_temperature = 10
t_31['dT'] = t_31.apply(lambda row: compute_cooling(row['dCVI'],T=grid_temperature, species=row['species']), axis=1)
t_31.tail()

,year,species,dbh,height,crownH,crownW,LAI,GPI,CVI,dLAI,dGPI,dCVI,dT
AgentID,,,,,,,,,,,,,
31,46,decidu,49.904146,20.838819,11.759976,10.735481,80.901747,1.182472,95.664082,432.121973,28.441797,583.467022,0.460602
31,47,decidu,50.255590,20.918409,11.791482,10.772873,81.469421,1.180340,96.161584,435.855785,28.210130,587.021400,0.462002
31,48,decidu,50.608610,20.998100,11.822557,10.810027,82.049072,1.178135,96.664875,439.668372,27.970652,590.617134,0.463415
31,49,decidu,50.947728,21.074413,11.851866,10.845337,82.615258,1.175959,97.152155,443.392397,27.734302,594.098483,0.464779
31,50,decidu,51.242808,21.140629,11.876938,10.875758,83.115740,1.174020,97.579541,446.684262,27.523686,597.151919,0.465972


In [23]:
grid_temperature = 34
t_31['dT'] = t_31.apply(lambda row: compute_cooling(row['dCVI'],T=grid_temperature, species=row['species']), axis=1)
t_31.tail()

,year,species,dbh,height,crownH,crownW,LAI,GPI,CVI,dLAI,dGPI,dCVI,dT
AgentID,,,,,,,,,,,,,
31,46,decidu,49.904146,20.838819,11.759976,10.735481,80.901747,1.182472,95.664082,432.121973,28.441797,583.467022,4.606016
31,47,decidu,50.255590,20.918409,11.791482,10.772873,81.469421,1.180340,96.161584,435.855785,28.210130,587.021400,4.620025
31,48,decidu,50.608610,20.998100,11.822557,10.810027,82.049072,1.178135,96.664875,439.668372,27.970652,590.617134,4.634153
31,49,decidu,50.947728,21.074413,11.851866,10.845337,82.615258,1.175959,97.152155,443.392397,27.734302,594.098483,4.647790
31,50,decidu,51.242808,21.140629,11.876938,10.875758,83.115740,1.174020,97.579541,446.684262,27.523686,597.151919,4.659719


In [24]:
grid_temperature = 48
t_31['dT'] = t_31.apply(lambda row: compute_cooling(row['dCVI'],T=grid_temperature, species=row['species']), axis=1)
t_31.tail()

,year,species,dbh,height,crownH,crownW,LAI,GPI,CVI,dLAI,dGPI,dCVI,dT
AgentID,,,,,,,,,,,,,
31,46,decidu,49.904146,20.838819,11.759976,10.735481,80.901747,1.182472,95.664082,432.121973,28.441797,583.467022,0.921203
31,47,decidu,50.255590,20.918409,11.791482,10.772873,81.469421,1.180340,96.161584,435.855785,28.210130,587.021400,0.924005
31,48,decidu,50.608610,20.998100,11.822557,10.810027,82.049072,1.178135,96.664875,439.668372,27.970652,590.617134,0.926831
31,49,decidu,50.947728,21.074413,11.851866,10.845337,82.615258,1.175959,97.152155,443.392397,27.734302,594.098483,0.929558
31,50,decidu,51.242808,21.140629,11.876938,10.875758,83.115740,1.174020,97.579541,446.684262,27.523686,597.151919,0.931944


## What do we have now?

Our DataFrame now contains 25 years of data concerning more than 30,000 trees in the Songpa district of Seoul.
Each has a unique AgentID, and by taking a cross-section of the data based on that AgentID, we can examine the evolution of various characteristics of the tree agents through the years of the simulation.

At that point, we can feed these arrays into the functions for leaf area index (LAI), gross canopy index (GCI), and crown volume index (CVI). These indices can then be used to track the evolution of a given agent 

We have multi-dimensional data, by year, and also by tree_agent. We could split the data first by year, comparing 0, and the year_in_question (whichever we decide to examine), then indexing on the grid_id, 

In [25]:
songpa_df = pd.read_csv("https://treesai-gus-public.s3.eu-west-3.amazonaws.com/inputs/songpa_yearly.csv")
# songpa_df = pd.read_csv("songpa_yearly.csv")
songpa_df.rename(columns={'Step': 'year'}, inplace=True)
songpa_df.set_index("AgentID", inplace=True)
songpa_df.tail(10)

,Unnamed: 0,year,species,dbh,height,crownH,crownW,canopy_overlap,cle,condition,dieback,biomass,seq,carbon,deroot,detrunk,mulched,burnt,coordinates,100_GRID_ID
AgentID,,,,,,,,,,,,,,,,,,,,
43621,945724,25,decidu,1.171978,4.133814,2.303374,2.023323,0,0.56,excellent,0.002496,0.338384,0.0,0.169192,0.0,0.0,0.0,0.0,"(2968, 2815)",112089
43622,945725,25,decidu,1.169297,4.133814,2.303374,2.023323,0,0.56,excellent,0.007161,0.336644,0.0,0.168322,0.0,0.0,0.0,0.0,"(615, 5251)",103185
43623,945726,25,decidu,1.843512,4.133814,2.303374,2.023323,0,0.56,excellent,0.008275,0.937750,0.0,0.468875,0.0,0.0,0.0,0.0,"(1986, 4644)",108125
43624,945727,25,decidu,1.581590,4.133814,2.303374,2.023323,0,0.56,excellent,0.006751,0.664248,0.0,0.332124,0.0,0.0,0.0,0.0,"(5367, 5117)",120622
43625,945728,25,decidu,1.159839,4.133814,2.303374,2.023323,0,0.56,excellent,0.000587,0.330547,0.0,0.165274,0.0,0.0,0.0,0.0,"(5266, 2769)",120644
43626,945729,25,decidu,1.294459,4.133814,2.303374,2.023323,0,0.56,excellent,0.000047,0.423203,0.0,0.211601,0.0,0.0,0.0,0.0,"(970, 5074)",104502
43627,945730,25,decidu,1.132638,4.133814,2.303374,2.023323,0,0.56,excellent,0.007452,0.313359,0.0,0.156679,0.0,0.0,0.0,0.0,"(402, 5167)",102199
43628,945731,25,decidu,1.506775,4.133814,2.303374,2.023323,0,0.56,excellent,0.008590,0.595625,0.0,0.297813,0.0,0.0,0.0,0.0,"(2385, 4602)",109771
43629,945732,25,decidu,1.226794,4.133814,2.303374,2.023323,0,0.56,excellent,0.008597,0.375043,0.0,0.187521,0.0,0.0,0.0,0.0,"(4848, 3998)",118988


In [26]:
N = 15 # Year in question
all_grid_ids = songpa_df['100_GRID_ID'].unique()
df_year_1 = songpa_df[songpa_df['year'] == 1]
df_year_N = songpa_df[songpa_df['year'] == N]

df_year_1.set_index('100_GRID_ID', inplace=True)
df_year_N.set_index('100_GRID_ID', inplace=True)

#TODO: Make a lookup for the temperature per grid_id
grid_temperature = 40#C

for grid in all_grid_ids:
    # Comparing this grid cell in year 1 and year N
    grid_df_1 = df_year_1.xs(grid)
    grid_df_N = df_year_N.xs(grid)
    
    # And use these to compute the change in LAI, GCI, CVI
    
    #TODO: There's surely a better way of handling this. How do the leaf area indices combine over a 100x100m grid?
    # Average the values in the LAI column
    avg_lai_1 = grid_df_1.apply(lambda row: compute_leaf_area_index(row['dbh'], row['height'],row['crownH'], row['crownW']), axis=1).mean()
    avg_gci_1 = grid_df_1.apply(lambda row: compute_gross_canopy_index(row['height'],row['crownH'], row['crownW']), axis=1).mean()
    avg_cvi_1 = compute_crown_volume_index(avg_lai_1, avg_gci_1)
    # print(avg_lai_1, avg_gci_1, avg_cvi_1)
    
    avg_lai_N = grid_df_N.apply(lambda row: compute_leaf_area_index(row['dbh'], row['height'],row['crownH'], row['crownW']), axis=1).mean()
    avg_gci_N = grid_df_N.apply(lambda row: compute_gross_canopy_index(row['height'],row['crownH'], row['crownW']), axis=1).mean()
    avg_cvi_N = compute_crown_volume_index(avg_lai_N, avg_gci_N)
    # print(avg_lai_N, avg_gci_N, avg_cvi_N)
    
    dLAI = compute_change(avg_lai_N, avg_lai_1)
    dGCI = compute_change(avg_gci_N, avg_gci_1)
    dCVI = compute_change(avg_cvi_N, avg_cvi_1)
    # print(dLAI, dGCI, dCVI)
    
    # grid_df_1["dT"] = grid_df_1.apply(lambda row: compute_cooling(row['dCVI'],T=grid_temperature, species=row['species']), axis=1)
    species = grid_df_1['species'].unique()
    dT = compute_cooling(dCVI,T=grid_temperature, species=species[0])
    print("Cooling for cell {} is {}".format(grid, dT))

Cooling for cell 106149 is 0.24065480102077733
Cooling for cell 105491 is nan
Cooling for cell 122921 is 0.8949778624212654
Cooling for cell 102199 is 0.6609112826222353
Cooling for cell 121940 is 0.856951954342229
Cooling for cell 102536 is 0.4676495202525727
Cooling for cell 121936 is 1.0179965566582059
Cooling for cell 115026 is 1.1803326182763874
Cooling for cell 112390 is 1.0424167227982786
Cooling for cell 122621 is 0.8939150341587475
Cooling for cell 114036 is 0.7161650500805937
Cooling for cell 106806 is nan
Cooling for cell 118659 is 1.1261619765672248
Cooling for cell 101550 is 0.6475283346631495
Cooling for cell 105497 is 0.1886173142978824
Cooling for cell 120955 is 0.8198970944462024
Cooling for cell 121632 is 0.5304979037330263
Cooling for cell 120641 is 1.2604202363238628
Cooling for cell 109109 is 0.4677179884412021
Cooling for cell 118319 is 0.8826521223157231
Cooling for cell 122598 is 0.5583749262331027
Cooling for cell 113402 is 0.6484769817081313
Cooling for cell 1

/var/folders/sq/279ztff55tg817zzbvzd4p0c0000gn/T/ipykernel_20331/2334204138.py:30: RuntimeWarning: invalid value encountered in sqrt
  return species_factor * np.sqrt(canopy_change_cvi/10) * cooling_multiplier(T)
/var/folders/sq/279ztff55tg817zzbvzd4p0c0000gn/T/ipykernel_20331/2334204138.py:30: RuntimeWarning: invalid value encountered in sqrt
  return species_factor * np.sqrt(canopy_change_cvi/10) * cooling_multiplier(T)
/var/folders/sq/279ztff55tg817zzbvzd4p0c0000gn/T/ipykernel_20331/2334204138.py:30: RuntimeWarning: invalid value encountered in sqrt
  return species_factor * np.sqrt(canopy_change_cvi/10) * cooling_multiplier(T)
/var/folders/sq/279ztff55tg817zzbvzd4p0c0000gn/T/ipykernel_20331/2334204138.py:30: RuntimeWarning: invalid value encountered in sqrt
  return species_factor * np.sqrt(canopy_change_cvi/10) * cooling_multiplier(T)


Cooling for cell 119643 is nan
Cooling for cell 116372 is 0.8515602546756449
Cooling for cell 115367 is 1.011500628272554
Cooling for cell 121944 is 1.0541349171725025
Cooling for cell 116370 is nan
Cooling for cell 105828 is 0.6800146033022855
Cooling for cell 115693 is nan
Cooling for cell 122267 is 0.7191956468925629
Cooling for cell 109113 is 0.6328858995109583
Cooling for cell 121299 is 0.732308738798658
Cooling for cell 102527 is 0.7646306976128737
Cooling for cell 116337 is 0.5535553272520203
Cooling for cell 102204 is 0.6949803523108281
Cooling for cell 107461 is 0.605473133225561
Cooling for cell 116014 is nan
Cooling for cell 118972 is 1.5926538655858604
Cooling for cell 122923 is 0.9342059410787507
Cooling for cell 108781 is nan
Cooling for cell 116012 is 1.0372203722745974
Cooling for cell 115352 is 0.6547097916592219
Cooling for cell 104505 is 1.0974021789287838
Cooling for cell 115340 is 2.012973483806389
Cooling for cell 110748 is 0.5436765389258925
Cooling for cell 1156

/var/folders/sq/279ztff55tg817zzbvzd4p0c0000gn/T/ipykernel_20331/2334204138.py:30: RuntimeWarning: invalid value encountered in sqrt
  return species_factor * np.sqrt(canopy_change_cvi/10) * cooling_multiplier(T)
/var/folders/sq/279ztff55tg817zzbvzd4p0c0000gn/T/ipykernel_20331/2334204138.py:30: RuntimeWarning: invalid value encountered in sqrt
  return species_factor * np.sqrt(canopy_change_cvi/10) * cooling_multiplier(T)
/var/folders/sq/279ztff55tg817zzbvzd4p0c0000gn/T/ipykernel_20331/2334204138.py:30: RuntimeWarning: invalid value encountered in sqrt
  return species_factor * np.sqrt(canopy_change_cvi/10) * cooling_multiplier(T)
/var/folders/sq/279ztff55tg817zzbvzd4p0c0000gn/T/ipykernel_20331/2334204138.py:30: RuntimeWarning: invalid value encountered in sqrt
  return species_factor * np.sqrt(canopy_change_cvi/10) * cooling_multiplier(T)
/var/folders/sq/279ztff55tg817zzbvzd4p0c0000gn/T/ipykernel_20331/2334204138.py:30: RuntimeWarning: invalid value encountered in sqrt
  return specie

Cooling for cell 115351 is nan
Cooling for cell 109436 is 0.6733766239612449
Cooling for cell 111733 is 0.7482435331212388
Cooling for cell 121278 is 0.768385020788472
Cooling for cell 108786 is nan
Cooling for cell 121971 is nan
Cooling for cell 113400 is 0.417922904031818
Cooling for cell 108777 is 0.8757475612772533
Cooling for cell 107793 is 0.08860438946496756
Cooling for cell 121635 is 1.414175526755008
Cooling for cell 112737 is 1.1313979783443597
Cooling for cell 102534 is nan
Cooling for cell 109443 is 0.45332060765874216
Cooling for cell 120956 is 0.9242898131498004
Cooling for cell 109115 is 0.3723062855688444
Cooling for cell 118676 is 0.6325358627464074
Cooling for cell 123271 is nan
Cooling for cell 121300 is 0.2134100049835255
Cooling for cell 112086 is 0.6870528239406797
Cooling for cell 103844 is nan
Cooling for cell 102528 is 1.2348542675279255
Cooling for cell 107463 is nan
Cooling for cell 114696 is 0.8475835277366504
Cooling for cell 104175 is 0.26701783712421334
C

...

/var/folders/sq/279ztff55tg817zzbvzd4p0c0000gn/T/ipykernel_20331/2334204138.py:30: RuntimeWarning: invalid value encountered in sqrt
  return species_factor * np.sqrt(canopy_change_cvi/10) * cooling_multiplier(T)
/var/folders/sq/279ztff55tg817zzbvzd4p0c0000gn/T/ipykernel_20331/2334204138.py:30: RuntimeWarning: invalid value encountered in sqrt
  return species_factor * np.sqrt(canopy_change_cvi/10) * cooling_multiplier(T)


Cooling for cell 105499 is 1.0228408520831578
Cooling for cell 121611 is 0.8532236418747141
Cooling for cell 114030 is 1.167104865483387
Cooling for cell 115999 is 0.6434204316259673
Cooling for cell 115339 is 2.2286125838624766
Cooling for cell 118675 is 1.4520848899532481
Cooling for cell 122944 is 1.2929956294837595
Cooling for cell 121631 is 0.9366424931693423
Cooling for cell 118005 is 0.9007892961819787
Cooling for cell 113403 is 1.1934855011742773
Cooling for cell 105162 is 0.45291914721178783
Cooling for cell 104843 is 0.7951370373862813
Cooling for cell 119988 is 0.4401294039351683
Cooling for cell 118349 is nan
Cooling for cell 116356 is 1.059445062842475
Cooling for cell 121941 is 0.9580364276459327
Cooling for cell 122949 is 0.9616465925918499
Cooling for cell 116013 is 0.6302243995822342
Cooling for cell 120642 is 0.7392525008677213
Cooling for cell 112064 is 0.6534752233438464
Cooling for cell 102865 is 0.9732108925950608
Cooling for cell 110749 is 0.36362266971761176
Coo

/var/folders/sq/279ztff55tg817zzbvzd4p0c0000gn/T/ipykernel_20331/2334204138.py:30: RuntimeWarning: invalid value encountered in sqrt
  return species_factor * np.sqrt(canopy_change_cvi/10) * cooling_multiplier(T)
/var/folders/sq/279ztff55tg817zzbvzd4p0c0000gn/T/ipykernel_20331/2334204138.py:30: RuntimeWarning: invalid value encountered in sqrt
  return species_factor * np.sqrt(canopy_change_cvi/10) * cooling_multiplier(T)
/var/folders/sq/279ztff55tg817zzbvzd4p0c0000gn/T/ipykernel_20331/2334204138.py:30: RuntimeWarning: invalid value encountered in sqrt
  return species_factor * np.sqrt(canopy_change_cvi/10) * cooling_multiplier(T)
/var/folders/sq/279ztff55tg817zzbvzd4p0c0000gn/T/ipykernel_20331/2334204138.py:30: RuntimeWarning: invalid value encountered in sqrt
  return species_factor * np.sqrt(canopy_change_cvi/10) * cooling_multiplier(T)
/var/folders/sq/279ztff55tg817zzbvzd4p0c0000gn/T/ipykernel_20331/2334204138.py:30: RuntimeWarning: invalid value encountered in sqrt
  return specie

Cooling for cell 111738 is 0.48351897893739054
Cooling for cell 113706 is 0.7603870481896767
Cooling for cell 118646 is 1.0103751718734582
Cooling for cell 112417 is 0.34223574139557544
Cooling for cell 111076 is nan
Cooling for cell 109768 is 0.8706341220913751
Cooling for cell 105490 is nan
Cooling for cell 115030 is 0.5998113170037745
Cooling for cell 118328 is 0.5876022605109193
Cooling for cell 116028 is 1.181379147007222
Cooling for cell 108451 is 0.3875406649548041
Cooling for cell 107133 is 0.9033130117007652
Cooling for cell 105160 is 0.3112269604435392
Cooling for cell 119634 is 0.49324982771888287
Cooling for cell 103190 is 1.0549965706100328
Cooling for cell 101219 is nan
Cooling for cell 110419 is 0.44525861532588534
Cooling for cell 116682 is 1.0508396751691766
Cooling for cell 119646 is 0.9795972633591274
Cooling for cell 117684 is 0.5965581067538597
Cooling for cell 120296 is 0.8556160047533418


TypeError: <lambda>() got an unexpected keyword argument 'axis'